# Import Packages

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import lxml
import time

# Read CSV

In [2]:
data = pd.read_csv("QA_final.csv")
data

,카테고리,question,answer
0,역사,고구려를 건국한 왕은?,주몽
1,역사,과거제를 처음 도입한 시대는?,고려시대
2,역사,훈민정음을 창제한 왕은?,세종대왕
3,역사,동학농민운동을 이끈 사람은?,전봉준
4,역사,문예 부흥이라는 중세 유럽에 일어난 운동은?,르네상스
...,...,...,...
95,인문학,인질이 인질범들에게 동조하는 비이성적 현상은?,스톡홀름 증후군
96,인문학,타인의 기대나 관심을 받을 경우 그러한 기대에 부응하여 긍정적인 형태를 보이게 되는 현상,피그말리온 효과
97,인문학,정신분석학의 창시자는?,프로이트
98,사회,캐나다의 수도는?,오타와


# Functions

In [3]:
# 질문을 question으로 받아 a 태그의 href 주소를 받고, return
# return value: href list
def find_ten_href(question):
    req = requests.get(f'https://search.naver.com/search.naver?where=post&sm=tab_jum&query={question}')
    html = req.text
    soup = bs(html, 'html.parser')
    
    blog_titles = soup.select('a.sh_blog_title')
    
    hrefs = []
    for title in blog_titles:
        hrefs.append(title.attrs['href'])            
    return hrefs

# 만약 naver 블로그면 iframe이 존재할 것이므로, iframe 탐색 후, iframe 주소 return
# return value: iframe url
def find_iframe_url(url):
    req = requests.get(url)
    html = req.text
    soup = bs(html, 'html.parser')
    
    iframe_located = soup.find('iframe')['src']
    iframe_site = 'https://blog.naver.com/'+iframe_located
    
    return iframe_site

# p tag에 텍스트들이 담겨 있으므로 모든 p 태그의 text를 갖고 온다.
# return value: p tag list
def get_p(url):
    blog_req = requests.get(url)
    html = blog_req.text
    soup = bs(html, 'html.parser')
    
    p = soup.select('p')
    
    return p

# p tag list에 대하여 하나의 텍스트로 합쳐버린다.
# return value: text
def to_text(target):
    to_text = lambda x: x.text
    target = list(map(to_text, target))
    target = ' '.join(target)
    return target

In [9]:
def main(question):
    # 10개의 href 습득
    ten_hrefs = find_ten_href(question)
#     success는 성공률 검사 위해
#     success = 0 
    text_list = []
    for href in ten_hrefs:
        try:
            # naver 게시물이면 iframe 탐색
            if 'naver' in href:
                iframe_url = find_iframe_url(href)
                ps = get_p(iframe_url)
            # 아니면 단순히 p태그 탐색 후 get
            else:
                ps = get_p(href)
            only_text = to_text(ps)
            text_list.append(only_text)
        except:
            pass
        
        
#         if len(only_text) != 0:
#             success += 1
#     print(f'{success}/10')
    print(text_list)
    return text_list

In [10]:
whole_start = time.time()
for question in data.question:
    start = time.time()
    print(question)
    main(question)
    end = time.time()
    print(end - start)
    print('---------')
    
print(time.time()-whole_start)

고구려를 건국한 왕은?
['오늘 찍은 사진을 퀵에디터로 남겨보세요. 삼국시대 (고구려, 백제, 신라)의 건국 왕을 알아보자! 오늘은 고구려, 백제, 신라 삼국을 건국한 왕을 알아보는 시간을 가져봅시다! \u200b 안녕하세요! 작은거인쌤입니다. ^^ \u200b 한국사를 공부하고 계신 여러분들을 위해 오늘은 고구려, 백제, 신라 3국을 건국한 건국 왕을 알아보려고 합니다. \u200b \'한국을 빛낸 100명의 위인들" 노래를 들어보면 누가 어느 나라를 건국했는지 간단하게 아실 수 있으실텐데요! \u200b 그 왕들이 나라를 세운 신화와 어떤 업적을 이루었는지 알아보도록 하겠습니다. \u200b 고구려의  시조왕 "동명왕" \u200b 먼저, 고구려의 건국 왕인 "동명왕" 입니다. \u200b \'주몽\' 이라는  이름이 더 와닿는 분들이 계실텐데요. 이 이름은 삼국사기에서는 동명왕의 활 솜씨가 뛰어나 붙여진 이름이라고 합니다! \u200b 동명왕의 형제였던 대소 왕자를 비롯한 형제들이 어려서부터 재주가 뛰어난 동명왕을 시기하고 질투해, 그는 부여를 떠날 수 밖에 없었습니다. \u200b 부여를 떠나면서 동명왕은 \'소서노\'라는 졸본지역의 강력한 부족장의 딸을 만나 힘과 재산을 받게되면서 B.C 37년 고구려를 건국하게 됩니다. \u200b 동명왕은 고구려를 건국한 뒤 먼저 세력을 확장하기 위해 정복 전쟁을 먼저 펼치기 시작했습니다. 고구려의 옆 나라였던 \'말갈\' 정복을 시작으로 점차 세력을 확장해 나가면서, \'소노, 계로, 관노, 절노, 순노\'의 5부족 나라를 건국할 수 있었습니다. \u200b 고구려는 이러한 탄탄한 국가 체제를 유지하며, 안정된 환경에서 법과 제도를 발전시킴으로써 점점 강한 힘을 가지게 됩니다. \u200b 백제의 시조왕 "온조왕" \u200b 다음으로 백제의 건국 왕인 "온조왕"입니다. \u200b 온조왕은 고구려를 세운 동명왕의 아들이었습니다. 하지만 동명왕이 후계 왕으로 배다른 형제였던 유리왕을 지목함으로써 어머니인 소서노와

['', "이 달의 결심, 실천일기를 퀵에디터로 기록해보세요. 한국사능력검정시험 46회 중급 기출문제 해설(조세 경제 역사 분야 등) - 소수림왕, 율령 반포, 과거제, 사비성, 영락, 관료전, 금관가야, 김수로왕, 상평통보, 이앙법, 벽란도 한국사능력검정시험 46회 중급 기출문제 해설(조세 경제 역사 분야 등) - 소수림왕, 율령 반포, 과거제, 사비성, 영락, 관료전, 금관가야, 김수로왕, 상평통보, 이앙법, 벽란도, 상평창, 인조반정, 광해군, 동의보감, 장용영, 노비안검법, 백두산정계비      정답 4 ① <해설> 소수림왕은 고구려의 제17대 왕(?~384). 재위 기간은 371~384년. 불교를 도입하고, 태학을 설립하였으며 율령을 반포하는 등 국가 체제를 정비하여 5세기 고구려 전성기의 기틀을 마련하였다.  ② 고려 958년(광종 9년)에 광종은 중국에서 귀화한 쌍기의 건의를 받아들여 과거제를 시행했다. ③ 백제의 성왕은 538년 수도를 사비(부여)로 옮겼다. ④ 연호 '영락'은 광개토대왕이 처음 쓴 것이다. ⑤ 관료전은 신라 중대에 중앙과 지방의 관리들에게 지급한 전지를 말한다.    정답 7 ② <해설> 김수로왕은 금관가야를 세운 임금이다. 재위 42~199.  ① 상평통보는 조선시대의 화폐이다. ③ 이앙법(모내기법)은 조선후기에 실시되었다. ④ 벽란도는 황해도 예성강 하류에 있던 고려 시대의 국제 무역항이다. ⑤ 상평창은 고려·조선 때의 물가 조절기관이다.      정답 25 ③ <해설> 인조반정은 1623년 이귀, 김유 등 서인 일파가 정변을 일으켜 광해군을 폐위시키고 인조를 왕위에 앉힌 사건이다.  ③ 동의보감은 광해군 때에 허준이 중국과 조선의 의서를 집대성하여 1610년에 저술한 의서이다. ① 장용영은 1784년(정조 8) 정조가 국왕의 호위를 맡아보던 숙위소를 폐지하고 새로운 금위체제에 따라 조직·개편한 국왕 호위군대다.  ④ 노비안검법은 고려 초기 광종 때 양인이었다가 노비가 된 사람을 조사하여 다시 양인이 될 수 있도록 조처한 법

["이 달의 결심, 실천일기를 퀵에디터로 기록해보세요. 세종대왕 훈민정음 누구를 위해 창제했을까 \u200b 세종대왕 훈민정음 누구를 위해 창제했을까 \u200b \u200b \u200b \u200b 조선의 네 번째 왕인 세종대왕에 대해 알아볼게요. 세종대왕은 우리나라 역사상 가장 위대한 왕이라는 평가를 받는데요. 그 이유는 훈민정음을 창제했기 때문이에요. \u200b \u200b 세종대왕 훈민정음에 대해 알아보기 전에 세종대왕에 대해 좀 더 자세하게 알아볼게요. 세종은 태종의 셋째 아들인데요. 원래는 태종이 그의 맏아들을 세자로 삼았답니다. \u200b \u200b 하지만 바르지 않은 행실로 세자 자리에서 쫓겨나게 되고, 세종에게 세자의 자리를 물려주게 되었어요. 세종은 왕이 되어 백성을 근본으로 하는 정치를 펼쳤는데요. \u200b \u200b 유능한 인재를 등용하고 황희와 같은 청렴한 재상을 가까이하며 나라를 다스렸답니다. 백성들의 조세부담을 덜기 위한 노력도 했던 인물이에요. \u200b \u200b \u200b \u200b \u200b \u200b 세종대왕 훈민정음뿐만 아니라 '농사직설'이라는 책도 펴냈는데요. 농사를 과학적으로 짓기 위해 많은 노력을 했답니다. 백성들에게는 유교 사상을 실천하도록 이야기하기도 했어요. \u200b \u200b 집현전을 설치해 인재를 육성하는데도 많은 노력을 했는데요. 세종 때는 과학 기술도 많이 발전했어요. 측우기와 자격루, 혼천의와 같은 것들이 만들어졌답니다. \u200b \u200b 세종대왕 하면 빼놓을 수 없는 게 바로 훈민정음인데요. 세종대왕은 왜 훈민정음이라는 글자를 만들었을까요? 그 이유는 바로 백성을 사랑했기 때문이에요. \u200b \u200b 조선시대 전까지는 한자로 글을 적었는데요. 우리말을 한자로 옮기는 게 쉽지 않았기 때문에 백성들이 한자를 배우고 익히는 건 쉽지 않았어요. \u200b \u200b \u200b \u200b \u200b \u200b 그러다 보니 당시에는 양반들만 글을 읽고 쓸 수 

["오늘 있었던 일을 퀵에디터로 기록해보세요. 동학농민혁명기념일 『동학농민운동을 이끈 녹두장군 전봉준』 <동학농민혁명기념일 『동학농민운동을 이끈 녹두장군 전봉준』> 한국사에서 빼놓을 수 없는 역사인 동학농민혁명은 비적, 역적으로 불리던 불명예를 벗고 지난 2004년 3월 국가로부터 공식적으로 '혁명'임을 인정 받게 되었습니다. 이후 작년 초 5월 11일을 동학농민혁명기념일로 공식적으로 지정하게 되었습니다. 오늘은 동학농민운동을 이끈 녹두장군 전봉준과 동학농민혁명의 진행 과정에 대해 알아보는 시간을 갖도록 하겠습니다. \u200b 동학농민운동을 이끈 녹두장군 전봉준 전봉준 장군(1855~1895)의 별명인 녹두는 어릴적 부터 몸이 작고 약해서 붙여졌다고 합니다. 그는 1890년 35세에 동학에 발을 들여놓았으며, 얼마 지나지 않아 교주인 최시형으로부터 고부 지방의 동학 접주로 임명되었습니다. \u200b 그는 효를 근본으로 삼으며 나랏일을 돕고 백성을 편안하게 하고자 했으며,  신분해방, 부패한 정치제도의 철폐, 토지 제도의 개혁, 일본침략 반대를 목표로 2차례에 걸쳐 동학농민혁명 운동을 주도하게 됩니다. \u200b <동학농민혁명 진행과정> 동학농민혁명은 1894년(고종 31년)에 동학교도인 전봉준이 중시이 되어 일으킨 반봉건·반외세 혁명 운동이라고 할 수 있습니다. 전체 운동은 크게 고부 봉기(제 1차), 전주·광주 궐기(제 2차)로 나뉘게 됩니다. \u200b - 제 1차 봉기(반봉건 운동) 전봉준 장군은 평등사상을 바탕으로 봉건사회를 반대하고 정부의 부패에 저항하기 위해 무장지역의 손화중과 손을 잡고 최시형의 탄생일(1894년 3월 21일)을 기하여 고부 백산에서 궐기를 하였습니다. 농민군은 관군을 무찌르며 삼남지방을 휩쓸고, 전주성까지 점령하게 되었습니다. 이에 조선 정부는 청에게 원군을 요청하였고, 나라 안팎의 위급상황을 인지한 전봉준 장군은 5월 8일 관군과 폐정개혁을 실시한다는 전주화약을 맺게 되었습니다. 이후 농민들은 각 고을에 자치기구인 집강소

KeyboardInterrupt: 